In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.\
         builder.\
         appName("Working with json").\
         master("local[*]").\
         getOrCreate()
        )
         

25/05/21 13:45:21 WARN Utils: Your hostname, oem-Lenovo-G50-80 resolves to a loopback address: 127.0.1.1; using 192.168.0.121 instead (on interface wlp3s0)
25/05/21 13:45:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 13:45:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [11]:
#read single line json
df_single = spark.read.format("json").load("data/input/single_line.json")

In [12]:
df_single.printSchema()

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [8]:
df_single.show()

+--------------------+-----------+--------+--------------------+
|             contact|customer_id|order_id|    order_line_items|
+--------------------+-----------+--------+--------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|
+--------------------+-----------+--------+--------------------+



In [9]:
#read multiple line json
df_multiple = spark.read.format("json").load("data/input/multiple_line.json")

In [10]:
df_multiple.show()

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [13]:
df_multiple1 = spark.read.format("json").option("multiline", True).load("data/input/multiple_line.json")

In [14]:
df_multiple1.show()

+--------------------+-----------+--------+--------------------+
|             contact|customer_id|order_id|    order_line_items|
+--------------------+-----------+--------+--------------------+
|[9000010000, 9000...|       C001|    O101|[{102.45, I001, 6...|
+--------------------+-----------+--------+--------------------+



In [15]:
df = spark.read.format("text").load("data/input/single_line.json")

In [16]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [17]:
df.show()

+--------------------+
|               value|
+--------------------+
|{"order_id":"O101...|
|                    |
+--------------------+



In [20]:
df.show(truncate = False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"order_id":"O101","customer_id":"C001","order_line_items":[{"item_id":"I001","qty":6,"amount":102.45},{"item_id":"I003","qty":2,"amount":2.01}],"contact":[9000010000,9000010001]}|
|                                                                                                                                                                                   |
+-----------------------------------------------------------------------------------------

In [23]:
_schema = "customer_id string, order_id string, contact array<long>"; 

In [24]:
df_schema = spark.read.format("json").schema(_schema).load("data/input/single_line.json")

+-----------+--------+--------------------+
|customer_id|order_id|             contact|
+-----------+--------+--------------------+
|       C001|    O101|[9000010000, 9000...|
+-----------+--------+--------------------+



In [28]:
_schema_new = "customer_id string, order_id string, contact array<string>, order_item_line array<struct<amount double,item_id string, qty long>>"; 

In [29]:
df_schema_new = spark.read.format("json").schema(_schema_new).load("data/input/single_line.json")

In [30]:
df_schema_new.show()

+-----------+--------+--------------------+---------------+
|customer_id|order_id|             contact|order_item_line|
+-----------+--------+--------------------+---------------+
|       C001|    O101|[9000010000, 9000...|           NULL|
+-----------+--------+--------------------+---------------+



In [32]:
#fucntion from_json to read from a column
from pyspark.sql.functions import from_json
_schema_= "customer_id string, order_id string, contact array<string>, order_item_line array<struct<amount double,item_id string, qty long>>"; 

df_extended = df.withColumn("parsed",from_json(df.value,_schema_))

In [34]:
df_extended.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_id: string (nullable = true)
 |    |-- contact: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- order_item_line: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- amount: double (nullable = true)
 |    |    |    |-- item_id: string (nullable = true)
 |    |    |    |-- qty: long (nullable = true)

